# Tropical Cyclone Rainfall Prediction: Autoencoder Model Implementation

This notebook demonstrates the implementation, training, and evaluation of an Autoencoder model for the task of translating Infrared (IR) satellite imagery to Passive Microwave Rainfall (PMR) estimations. The core idea is to learn a compressed representation of the input IR image and then reconstruct the corresponding PMR image.

We will cover:
1.  **Setup and Global Parameters**: Importing necessary libraries and defining constants.
2.  **Data Preprocessing**: Functions for loading, augmenting, resizing, and normalizing image pairs.
3.  **Dataset Preparation**: Creating TensorFlow `tf.data.Dataset` objects for training and testing.
4.  **Autoencoder Architecture**: Defining the custom Autoencoder model using a U-Net-like structure.
5.  **Model Instantiation and Training**: Setting up the model, defining callbacks, and initiating the training process.
6.  **Model Testing and Visualization**: Evaluating the trained model and visualizing its predictions against ground truth images.

---

### 1. Setup and Global Parameters

We begin by importing all necessary libraries, including TensorFlow for building the neural network, NumPy for numerical operations, Matplotlib for plotting, and Pandas for data manipulation. We also set up global parameters like image dimensions, batch sizes, and activation functions for experimentation. `scienceplots` is used for scientific-style plots.

In [ ]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.callbacks import TensorBoard, CSVLogger
from tensorflow.keras.models import Model, load_model
from keras.datasets import mnist # This import seems unused in the provided code
from itertools import product # This import seems unused in the provided code
from tensorflow.data import AUTOTUNE
import os
import math
import random # This import seems unused in the provided code
import scienceplots

# Apply a scientific plotting style for better visualization
plt.style.use('science')

# Define hyperparameter choices for potential experimentation (though only one choice is run here)
batch_sizes = (4, 8, 16, 32, 64)
last_activations = ("linear", "relu", "sigmoid", "tanh")

# Chosen parameters for the current run
chosen_parameters = [64, "linear"] 
num_epochs = 20

print(f"Running test with parameters: {chosen_parameters}")

# Assign chosen parameters to variables for model configuration
batch_size = int(chosen_parameters[0])
last_activation = chosen_parameters[1]

# Define dataset buffer size and batch size
DATASET_BUFFER_SIZE = 1000
DATASET_BATCH_SIZE = batch_size

# Define target image dimensions
IMG_WIDTH = 256
IMG_HEIGHT = 256


---

### 2. Data Preprocessing

The `preprocess_pair` function handles loading, augmenting, resizing, and normalizing the image data. Each input JPG file is expected to contain two images side-by-side: the "real" (PMR) image on the left and the "input" (IR) image on the right.

-   **Loading and Splitting**: Reads the JPEG image as grayscale (`channels=1`) and splits it into `real` (left half) and `inp` (right half).
-   **Augmentation (Training Only)**: During training, `RandomRotation`, `RandomZoom`, `RandomContrast`, and `RandomBrightness` augmentations are applied. Crucially, the input and real images are concatenated *before* augmentation and then split back to ensure identical transformations.
-   **Resizing**: Both images are resized to `IMG_WIDTH` x `IMG_HEIGHT` using bilinear interpolation and reflective padding to avoid artifacts.
-   **Normalization**: Z-score normalization is applied to each image independently to standardize pixel intensities.

In [ ]:
# 1) Specific augmentation layers defined as a Sequential model
data_augment = tf.keras.Sequential([
    # Random rotations between 0 and 360 degrees (factor 1.0 means full circle)
    layers.RandomRotation(factor=1.0, fill_mode='reflect'),
    # Light zoom in/out (±10%)
    layers.RandomZoom(0.1, 0.1, fill_mode='reflect'),
    # Light contrast variations
    layers.RandomContrast(0.05),
    # Light brightness variations
    layers.RandomBrightness(0.05),
])

def preprocess_pair(path, training):
    # a) Load and split the image
    # Decode JPEG to grayscale (1 channel for intensity)
    img = tf.io.decode_jpeg(tf.io.read_file(path), channels=1)
    w = tf.shape(img)[1] // 2 # Calculate half width to split
    
    # "real" (PMR) is the left half, "inp" (IR) is the right half
    real = img[:, :w, :]
    inp  = img[:, w:, :]

    # b) Apply augmentation only during training
    if training:
        # Concatenate to apply identical transformations to both images
        pair = tf.concat([inp, real], axis=2)  # shape (H, W, 2)
        pair = data_augment(pair) # Apply defined augmentations
        inp, real = tf.split(pair, num_or_size_splits=2, axis=2) # Split back

    # c) Resize with bilinear interpolation + reflective padding to avoid artifacts
    inp  = tf.image.resize(inp,  [IMG_WIDTH, IMG_HEIGHT], method='bilinear')
    real = tf.image.resize(real, [IMG_WIDTH, IMG_HEIGHT], method='bilinear')

    # d) Z-score normalization per image (standardize pixel values)
    inp_mean, inp_var = tf.nn.moments(inp, axes=[0,1])
    real_mean, real_var = tf.nn.moments(real, axes=[0,1])
    inp  = (inp  - inp_mean)  / tf.sqrt(inp_var  + 1e-6) # Add epsilon for numerical stability
    real = (real - real_mean) / tf.sqrt(real_var + 1e-6)

    return inp, real


---

### 3. Dataset Preparation

The `make_ds` function creates optimized TensorFlow `Dataset` objects for efficient data loading during training and testing.

-   **List Files**: Lists all image files matching the given pattern.
-   **Shuffling (Training Only)**: Shuffles the dataset for training to ensure randomness and prevent overfitting to data order.
-   **Mapping**: Applies the `preprocess_pair` function to each file in parallel using `AUTOTUNE` for optimized performance.
-   **Batching**: Batches the processed images into `DATASET_BATCH_SIZE`. `drop_remainder=True` ensures all batches have the same size.
-   **Prefetching**: `prefetch(AUTOTUNE)` allows the data pipeline to fetch new batches in the background while the model is training on the current batch, minimizing idle time.

In [ ]:
def make_ds(pattern, training):
    ds = tf.data.Dataset.list_files(pattern)
    if training:
        ds = ds.shuffle(DATASET_BUFFER_SIZE) # Shuffle for training dataset
    ds = ds.map(lambda p: preprocess_pair(p, training),
                num_parallel_calls=AUTOTUNE) # Map preprocessing function in parallel
    ds = ds.batch(DATASET_BATCH_SIZE, drop_remainder=True) # Batch data
    ds = ds.prefetch(AUTOTUNE) # Prefetch for optimized pipeline
    return ds

# Create training and test datasets
train_dataset = make_ds('./data/TCIRRP/train1k/*.jpg', training=True)
test_dataset  = make_ds('./data/TCIRRP/test0.1k/*.jpg',  training=False) # Smaller test set for validation
test_dataset_full = make_ds('./data/TCIRRP/test/*.jpg', training=False) # Full test set for final evaluation


---

### 4. Autoencoder Architecture

The `Autoencoder` class defines the neural network architecture. It consists of an `encoder` (to compress the input) and a `decoder` (to reconstruct the image). This architecture is influenced by the U-Net design, focusing on hierarchical feature learning.

-   **Encoder**:
    -   Takes a `(IMG_WIDTH, IMG_HEIGHT, 1)` grayscale image as input.
    -   Comprises three `Conv2D` layers. Each layer uses a 3x3 kernel, `strides=2` (for downsampling), `padding='same'`, `BatchNormalization` (for stable training), and `ReLU` activation.
    -   The number of filters increases with depth: 32 -> 64 -> 128, capturing increasingly complex features.
    -   The output is `Flatten`ed and passed through a `Dense` layer to create the `LATENT_DIMENSIONS` (32) bottleneck.

-   **Decoder**:
    -   Takes the `LATENT_DIMENSIONS` vector as input.
    -   Starts with a `Dense` layer to project the latent vector back to the dimensions suitable for reshaping into a 3D tensor (`self.bw * self.bh * self.bc`).
    -   `Reshape`s the output to the bottleneck spatial dimensions.
    -   Uses two `Conv2DTranspose` layers (also known as deconvolution) for upsampling. These layers mirror the encoder's downsampling, increasing spatial dimensions with `strides=2`.
    -   The number of filters decreases: 64 -> 32. Each layer is followed by `BatchNormalization` and `ReLU`.
    -   The final `Conv2DTranspose` layer outputs a single channel image of the original size, with `last_activation` (e.g., 'linear' for continuous output) applied.

In [ ]:
LATENT_DIMENSIONS = 32 # Dimension of the compressed latent space
    
class Autoencoder(Model):
    def __init__(self, **kwargs):
        super(Autoencoder, self).__init__(**kwargs)
        
        # Encoder: Applies convolutional layers to compress input to latent space
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 1)), # Input shape: grayscale image
            
            # First Convolutional Block: Downsamples and increases filters
            layers.Conv2D(32, 3, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(), # Normalizes activations for stable training
            layers.Activation('relu'),   # Rectified Linear Unit activation
            
            # Second Convolutional Block
            layers.Conv2D(64, 3, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            
            # Third Convolutional Block
            layers.Conv2D(128, 3, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
    
            layers.Flatten(), # Flattens the 3D feature maps to a 1D vector
            layers.Dense(LATENT_DIMENSIONS, activation='relu'), # Maps to the latent space
        ])
        
        # Calculate bottleneck dimensions for reshaping in the decoder
        self.bw = math.ceil(IMG_WIDTH / 2**3)   # Width after 3 stride-2 convolutions
        self.bh = math.ceil(IMG_HEIGHT / 2**3)  # Height after 3 stride-2 convolutions
        self.bc = 128                           # Channels at the bottleneck (from last encoder layer)
                    
        # Decoder: Reconstructs image from latent space
        self.decoder = tf.keras.Sequential([
            layers.Input(shape=(LATENT_DIMENSIONS,)), # Input is the latent vector
            
            # Dense layer to expand latent vector before reshaping
            layers.Dense(self.bw * self.bh * self.bc, use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            
            layers.Reshape((self.bw, self.bh, self.bc)), # Reshape to 3D tensor for transposed convolutions
            
            # First Transposed Convolutional Block: Upsamples and decreases filters
            layers.Conv2DTranspose(64, 3, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            
            # Second Transposed Convolutional Block
            layers.Conv2DTranspose(32, 3, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            
            # Final Transposed Convolutional Layer: Outputs the reconstructed image (1 channel)
            layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation=last_activation),
        ])
    
    # Defines the forward pass of the autoencoder
    def call(self, input_data):
        encoded = self.encoder(input_data) # Encode the input
        decoded = self.decoder(encoded)    # Decode the latent representation
        return decoded


---

### 5. Model Instantiation and Training

This section handles the setup for model training, including loading previous checkpoints, compiling the model, and defining callbacks for monitoring and saving.

-   **Model Pathing**: Defines paths for saving and loading the model based on hyperparameters.
-   **Loading Checkpoint**: Checks if a pre-trained model exists at `input_model_path`. If so, it loads it and evaluates it on a small test set.
-   **Model Compilation**: If no model is loaded, a new `Autoencoder` instance is created and compiled. The `adam` optimizer is used, `MeanSquaredError` (MSE) as the loss function (suitable for regression tasks like image reconstruction), and `Accuracy` as a metric (though MSE itself is often the primary metric for regression).
-   **Callbacks**:
    -   `ModelCheckpoint`: Saves the model with the best validation loss.
    -   `TensorBoard`: Logs training metrics and graph information for visualization in TensorBoard.
-   **Training**: The `fit` method starts the training process using the `train_dataset` and validates on `test_dataset`.

In [ ]:
## Model instantiation

# Define paths for saving/loading the model and TensorBoard logs
input_epoch = 0 # Starting epoch for loading a pre-trained model (if any)
input_model_path = f"0.01k-{input_epoch}Epochs-32Lat-{batch_size}Batch-{last_activation}.keras"
output_epoch = num_epochs # Total number of epochs for training
output_model_path = f"0.01k-{output_epoch}Epochs-32Lat-{batch_size}Batch-{last_activation}.keras"
log_dir = f"logs/fit/v2-autoencoder-0.01k-32Lat-{last_activation}-{batch_size}Batch"

# Check if a pre-trained model exists
if os.path.exists(input_model_path):
    print("\033[92mModel loaded. With evaluation:\033[0m")
    # Load the model with custom_objects to recognize the Autoencoder class
    autoencoder = load_model(input_model_path, custom_objects={'Autoencoder': Autoencoder})
    autoencoder.evaluate(test_dataset, verbose=2) # Evaluate the loaded model
    print(autoencoder.optimizer.get_config()) # Print optimizer configuration

else:
    print("Model not loaded")
    autoencoder = Autoencoder() # Create a new Autoencoder instance
    # Compile the model: Adam optimizer, Mean Squared Error loss, and Accuracy metric
    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError(), metrics=[tf.keras.metrics.Accuracy()])

# You can uncomment these lines to view the summary of encoder and decoder architectures
# autoencoder.encoder.summary()
# autoencoder.decoder.summary()

## Model training

# ModelCheckpoint callback: Saves the model with the best validation loss
save_callback = tf.keras.callbacks.ModelCheckpoint(filepath=output_model_path, verbose=2, monitor='val_loss')

# TensorBoard callback: Logs training progress for visualization
tensorboard_cb = TensorBoard(
    log_dir=log_dir,
    histogram_freq=5,      # Save histograms of weights every 5 epochs
    write_graph=True,      # Save the model graph definition
)

# Train the autoencoder model
autoencoder.fit(
    train_dataset,
    epochs=output_epoch,       # Total number of epochs to train for
    initial_epoch=input_epoch, # Starting epoch (useful for resuming training)
    validation_data=test_dataset, # Data for validation during training
    callbacks=[save_callback, tensorboard_cb], # List of callbacks to use
)


---

### 6. Model Testing and Visualization

After training, the model's performance is evaluated on the full test dataset. Additionally, a visualization section generates predictions for a few samples from both the training and testing sets, allowing for a visual comparison of the input, predicted, and ground truth images.

In [ ]:
## Model Testing

print("\n\n\n")

# Evaluate the model on the full test dataset
print("Final evaluation:")
# Initialize results list (assuming it's defined elsewhere, e.g., `results = []`)
results = [] # Initialize results list here for standalone execution
ev = autoencoder.evaluate(test_dataset_full, verbose=2)
# Append results (batch size, last activation, and loss value)
results.append((batch_size, last_activation, ev[0])) 

print("Final evaluation result:", ev)
    
print(results)

# Number of samples to visualize from each dataset
num_tests = 5

# Loop through training and testing datasets for visualization
for name, dataset in [("Training set", train_dataset), ("Testing set", test_dataset)]:
    # 1) Extract a few samples
    x_input = []
    x_real  = []
    for input_batch, real_batch in dataset.take(num_tests):
        x_input.extend(input_batch)
        x_real.extend(real_batch)
    # Stack samples into tensors for batch processing
    x_input = tf.stack(x_input)
    x_real  = tf.stack(x_real)

    # 2) Encode and decode the input images to get predictions
    encoded_imgs = autoencoder.encoder(x_input).numpy()
    decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

    # 3) Plot the original input, predicted output, and target real image
    plt.figure(figsize=(6, num_tests * 2)) # Adjust figure size based on number of samples
    plt.suptitle(name, fontsize=16) # Set title for the plot set (Training/Testing set)
    
    for i in range(num_tests):
        # Column 1: Original input image (IR)
        ax = plt.subplot(num_tests, 3, i*3 + 1)
        plt.imshow(x_input[i].numpy(), cmap="gray")
        ax.set_title("Input (IR)")
        ax.axis("off") # Hide axes

        # Column 2: Reconstructed/Predicted image (PMR estimate)
        ax = plt.subplot(num_tests, 3, i*3 + 2)
        plt.imshow(decoded_imgs[i], cmap="gray")
        ax.set_title("Prediction (PMR)")
        ax.axis("off")

        # Column 3: "Real" target image (Ground Truth PMR)
        ax = plt.subplot(num_tests, 3, i*3 + 3)
        plt.imshow(x_real[i].numpy(), cmap="gray")
        ax.set_title("Target (PMR)")
        ax.axis("off")
                
    # Adjust layout to prevent overlap
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show() # Display all generated plots
